In [1]:
!pip install adversarial-robustness-toolbox

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [106]:
data_path = './recruitmentdataset-2022-1.3.csv'

In [107]:
df = pd.read_csv(data_path)
df.head(10)

,Id,gender,age,nationality,sport,ind-university_grade,ind-debateclub,ind-programming_exp,ind-international_exp,ind-entrepeneur_exp,ind-languages,ind-exact_study,ind-degree,company,decision
0,x8011e,female,24,German,Swimming,70,False,False,False,False,1,True,phd,A,True
1,x6077a,male,26,German,Golf,67,False,True,False,False,2,True,bachelor,A,False
2,x6006e,female,23,Dutch,Running,67,False,True,True,False,0,True,master,A,False
3,x2173b,male,24,Dutch,Cricket,70,False,True,False,False,1,True,master,A,True
4,x6241a,female,26,German,Golf,59,False,False,False,False,1,False,master,A,True
5,x9063d,female,26,Dutch,Chess,63,False,False,False,False,1,True,bachelor,A,True
6,x5785d,female,27,Dutch,Tennis,63,True,True,False,False,2,True,bachelor,A,False
7,x8767c,female,22,Dutch,Swimming,71,False,True,False,False,1,True,master,A,True
8,x6541b,female,28,Dutch,Football,65,True,False,False,True,3,False,bachelor,A,False
9,x3890b,male,24,Dutch,Football,55,True,False,False,True,3,False,master,A,True


In [108]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
df = df.drop(['Id'], axis=1)

df = pd.get_dummies(df, drop_first=True)

# Display the first few rows of the dataframe
df.head(10)

,age,ind-university_grade,ind-debateclub,ind-programming_exp,ind-international_exp,ind-entrepeneur_exp,ind-languages,ind-exact_study,decision,gender_male,...,sport_Golf,sport_Rugby,sport_Running,sport_Swimming,sport_Tennis,ind-degree_master,ind-degree_phd,company_B,company_C,company_D
0,24,70,False,False,False,False,1,True,True,False,...,False,False,False,True,False,False,True,False,False,False
1,26,67,False,True,False,False,2,True,False,True,...,True,False,False,False,False,False,False,False,False,False
2,23,67,False,True,True,False,0,True,False,False,...,False,False,True,False,False,True,False,False,False,False
3,24,70,False,True,False,False,1,True,True,True,...,False,False,False,False,False,True,False,False,False,False
4,26,59,False,False,False,False,1,False,True,False,...,True,False,False,False,False,True,False,False,False,False
5,26,63,False,False,False,False,1,True,True,False,...,False,False,False,False,False,False,False,False,False,False
6,27,63,True,True,False,False,2,True,False,False,...,False,False,False,False,True,False,False,False,False,False
7,22,71,False,True,False,False,1,True,True,False,...,False,False,False,True,False,True,False,False,False,False
8,28,65,True,False,False,True,3,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,24,55,True,False,False,True,3,False,True,True,...,False,False,False,False,False,True,False,False,False,False


In [109]:
X = df.drop(['decision'], axis=1)
y = df['decision']

# One-hot encode the target variable (one column for each class)
y = pd.get_dummies(y)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [111]:
from art.estimators.classification import SklearnClassifier
model = SVC(C=1.0, kernel='rbf', gamma='auto')

# Create a classifier from the trained model
classifier = SklearnClassifier(model=model, clip_values=(0, 1))

print(X_train.shape, y_train.shape)
y_train = np.reshape(y_train, (-1, 2))
print(y_train)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print('Accuracy on test data:', accuracy_score(y_test, y_pred))


(3200, 24) (3200, 2)
[[ True False]
 [False  True]
 [ True False]
 ...
 [ True False]
 [False  True]
 [ True False]]
Accuracy on test data: 0.8375


In [ ]:
from art.attacks.evasion import FastGradientMethod


# Create the adversarial attack
attack = FastGradientMethod(estimator=classifier, eps=0.6)

# Generate the adversarial test examples
X_test_adv = attack.generate(x=X_test)

# Evaluate the classifier on the adversarial test examples
y_pred_adv = classifier.predict(X_test_adv)
print('Accuracy on adversarial test data:', accuracy_score(y_test, y_pred_adv))

In [127]:
#Metrics to study the robustness of the model (CLEVER, loss sensitivity, empirical robustness)
from art.metrics import clever, loss_sensitivity, empirical_robustness
clever = clever(classifier, X_test, y_test, batch_size=100, radius=2, norm=2)
print('CLEVER score:', clever)

loss_sensitivity = loss_sensitivity(classifier, X_test, y_test)
print('Loss sensitivity:', loss_sensitivity)


emp_robustness = empirical_robustness(classifier, X_test, 'fgsm')
print('Empirical robustness:', emp_robustness)

Loss sensitivity: 1.2213746
Empirical robustness: 0.7117552534218877


In [116]:
# Implement another attack
from art.attacks.evasion import ProjectedGradientDescent

# Create the adversarial attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.5)

# Generate the adversarial test examples
X_test_adv = attack.generate(x=X_test)

# Evaluate the classifier on the adversarial test examples
y_pred_adv = classifier.predict(X_test_adv)
print('Accuracy on adversarial test data:', accuracy_score(y_test, y_pred_adv))

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 